# Restaurant Management System

In [1]:
class Restaurant:
    
    #restaurant_num = 0
    order_num = 0
    
    def __init__(self, name, address, id_item_map, name_id_map, tables, order_map):
        Restaurant.order_num += 1
        self.id = Restaurant.order_num
        
        self.name = name.strip()
        self.address = address
        self.tables = int(tables)
        
        self.items = id_item_map
        self.name_id_map = name_id_map
        self.previous_orders = {}
        self.orders = order_map
    
    def get_items(self):
        #list(filter(lambda val : val[1].is_available == True, id_item_map.items()))
        return [item for item in list(self.items.values()) if item.is_available]
        
    def get_status(self, order_id):
        return self.orders[order_id].status
        
    def remove_tax(self, order_id):
        self.order[order_id].tax_per = 0
    
    def get_bill(self, order_id, payment):
        
        order = self.orders[order_id]
        total_amount -= (order.total_amount*order.discount_perc)/100
        total_amount += (self.total_amount*self.tax_perc)/100
        
        self.orders[order_id].total_amount = total_amount
        self.orders[order_id].payment = payment
        self.orders[order_id].status = Status.CLOSED
        
        return self.total_amount
        
    def get_item(self, order_id, item_id):
        return self.items[id]
    
    def change_price(self, id, new_price):
        if id in self.items:
            self.items[id].price = float(new_price)
            return True
        else:
            return False
    
    def change_name(self, id, new_name):
        
        if id in self.items:
            old_name = self.items[id].name
            self.items[id].name = new_name.strip()
            
            # change the item name in name_id map also
            self.name_id_map.pop(old_name)
            self.name_id_map[new_name.strip()] = id
            return True
        
        else:
            return False
    
    def mark_unavailable(self, id):
        if id in self.items:
            self.items[id].is_available = False
            return True
        else:
            return False
    
    def mark_available(self, id):
        self.items[id].is_available = True
        
    def __str__(self):
        return f'Restaurant [id : {self.id}, name : {self.name}, address : {self.address}, items : {self.items}]'
    
    def __repr__(self):
        return f'Restaurant("id" : {self.id}, "name" : {self.name}, "address" : {self.address}, "items" : {self.items})'

In [2]:
id_item_map = {}
name_id_map = {}

class Item:
    
    item_num=0
    
    def __init__(self, name, price, is_available=True):
        self.name = name.strip()
        self.price = float(price)
        Item.item_num += 1
        self.id = Item.item_num
        self.is_available = is_available
        id_item_map[self.id] = self
        name_id_map[self.name] = self.id
    
    def __str__(self):
        return f'Item [id : {self.id}, name : {self.name}, price : {self.price}, available : {self.is_available}]'
    
    def __repr__(self):
        return f'Item("id" : {self.id}, "name" : {self.name}, "price" : {self.price}, "available" : {self.is_available})'

In [3]:
class OrderType:
    DINEIN = 'DineIn'
    UBEREATS = 'UberEats'
    GRUBHUB = 'GrubHub'
    DOORDASH = 'DoorDash'
    CARRYOUT = 'CarryOut'

class Status:
    OPEN = 'Open'
    PREPARING = 'Preparing'
    PENDING = 'Pending'
    CLOSED = 'Closed'

class Payment:
    CASH = 'Cash'
    CREDIT = 'Credit'
    DEBIT = 'Debit'
    APPLEPAY = 'ApplePay'
    GOOGLEPAY = 'GooglePay'

In [7]:
from datetime import date
from datetime import datetime

order_map = {}
    
class Order:
    
    order_num = 0
    
    # default date is current date
    def __init__(self, name, item_id_quantity, order_type, order_date=date.today()):
        
        self.name = name
        self.date = datetime.strptime(order_date, "%Y-%m-%d")
        
        # this is to start the order number from 1 on every day, 
        if (self.date, 1) not in order_map:
            Order.order_num  = 1
        else:
            Order.order_num += 1
            
        self.order_id = Order.order_num
    
        self.order_type = order_type
        self.items = item_id_quantity
        self.tax_perc = 4
        self.payment = None
        self.discount_perc = 0
        self.total_amount = 0
        
        # Online orders were prepaid
        if order_type in set(OrderType.DINEIN, OrderType.CARRYOUT):
            self.status = Status.OPEN
        else:
            self.status = Status.CLOSED
            
        for item_id, quant in items_quantity.items():
            item = id_item_map[item_id]
            self.total_amount += quant*item.price
        
        # tuple that stores (ordered date, order id) -> Order
        order_map[(self.date, self.order_id)] = self
    
    def add_item(self, item_id, quant=1):
        self.items[item_id] = quant + self.items.get(item_id,0)
        self.total_amount += quant * self.items[item_id].price
        
    def remove_item(self, item_id, quant=1):
        self.items[item_id] -= quant
        self.total_amount -= quant * self.items[item_id].price
    
    def apply_discount(self, discount_perc):
        self.discount_perc = discount_perc
    
    def get_status(self):
        return self.status
        
    def remove_tax(self):
        self.tax_per = 0
    
    def get_bill(self, payment):
        
        self.total_amount -= (self.total_amount*discount_perc)/100
        self.total_amount += (self.total_amount*self.tax_perc)/100
        self.status = Status.CLOSED
        self.payment = payment
        
        return self.total_amount    